# Simulação de Monte Carlo


## Explicação

### 🎯 O que é a Simulação de Monte Carlo?

A Simulação de Monte Carlo é uma técnica estatística que utiliza **repetidas gerações aleatórias** para modelar a evolução futura de uma variável. este caso, a rentabilidade de uma carteira de ativos financeiros.

Cada simulação representa um **cenário possível** para a trajetória dos retornos ao longo do tempo, considerando a média, a volatilidade e a correlação entre os ativos.

---

### ⚙️ Como Funciona o Processo?

1. **Configuração inicial:**
   - Selecionamos ativos (ex.: PETR4, VALE3, MSFT34 etc.) e seus pesos na carteira.
   - Calculamos os **retornos médios diários** e a **matriz de covariância** dos ativos.

2. **Geração de Cenários:**
   - Utilizamos a média de retornos e a estrutura de correlação (via decomposição de Cholesky) para gerar **retornos sintéticos**.
   - Simulamos **100.000 caminhos** de preços possíveis para a carteira ao longo de **252 dias úteis** (aproximadamente 1 ano).

3. **Evolução do Capital:**
   - Para cada simulação, multiplicamos os retornos diários cumulativos pelo **capital inicial** investido (ex.: R$ 10.000).

4. **Análises extraídas:**
   - **Retorno mediano** e **percentis** (5%, 1%) do capital final.
   - **Value at Risk (VaR)** — perdas máximas esperadas em níveis de 95% e 99% de confiança.
   - **Conditional Value at Risk (CVaR)** — perdas médias esperadas nos piores cenários.
   - **Probabilidade de lucro** após 1 ano.
   - **Desvio padrão** (volatilidade) dos retornos finais.
   - **Sharpe Ratio** — retorno ajustado pelo risco.

---

### 📊 O Que Podemos Interpretar da Simulação?

- **Previsão de Cenários Possíveis:**  
  Avaliamos a distribuição dos resultados possíveis da carteira ao final de 1 ano.

- **Estimativa de Risco:**  
  Medimos o risco através do VaR e do CVaR, identificando as potenciais perdas máximas em diferentes níveis de confiança.

- **Análise de Retornos:**  
  Calculamos o retorno médio esperado, além do melhor e pior cenário entre as simulações.

- **Ajuste Risco/Retorno:**  
  O **Sharpe Ratio** indica se o retorno compensaria o risco corrido.

- **Distribuição dos Montantes:**  
  O histograma final mostra graficamente a frequência dos diferentes montantes finais alcançados pelas simulações.

---

### 📋 Conclusão

A Simulação de Monte Carlo é uma ferramenta poderosa para:
- **Testar a robustez** da carteira diante de incertezas do mercado.
- **Avaliar riscos extremos** que não são capturados apenas pela média dos retornos.
- **Ajudar na tomada de decisão** de investimentos, permitindo entender melhor o perfil de risco da carteira.

Ela não prevê o futuro, mas **quantifica incertezas** usando estatística!

---


## Interpretação Prática dos Resultados da Simulação

### ✅ Exemplo de Interpretação

#### 🔹 Capital inicial:
- **R$ 10.000,00**

#### 🔹 Retorno Mediano:
- **R$ 11.500,00** após 1 ano.
- 👉 Com **50% de chance**, seu montante estará acima desse valor.

#### 🔹 Value at Risk (VaR):

| Nível de Confiança | VaR (Valor Absoluto) | Interpretação |
|:---|:---|:---|
| 95% | R$ 1.500 | Há 5% de chance de perder **mais de R$ 1.500** no período. |
| 99% | R$ 2.500 | Há 1% de chance de perder **mais de R$ 2.500** no período. |

**Resumo prático do VaR:**
> "Com 95% de confiança, espero perder no máximo R$ 1.500 em 1 ano, caso as condições de mercado se mantenham normais."

#### 🔹 Conditional Value at Risk (CVaR):

| Nível de Confiança | CVaR (Média das Piores Perdas) | Interpretação |
|:---|:---|:---|
| 95% | R$ 1.700 | Em caso de perdas piores que o VaR 95%, a média das perdas será **R$ 1.700**. |
| 99% | R$ 2.800 | Em caso de perdas piores que o VaR 99%, a média das perdas será **R$ 2.800**. |

**Resumo prático do CVaR:**
> "Se ultrapassarmos o VaR de 95%, a perda média será ainda maior: cerca de R$ 1.700."

#### 🔹 Probabilidade de Lucro:
- **Ex.: 80% dos cenários** simulados resultaram em **lucro** ao final de 1 ano.

---

### 🧠 Em resumo:

- **Monte Carlo** nos ajuda a entender **cenários possíveis**, **riscos extremos** e **potencial de ganho**.
- As métricas **VaR** e **CVaR** são essenciais para quem quer **controlar o risco** da carteira, além de buscar apenas retorno.
- **Não é previsão de futuro**, mas uma poderosa **quantificação das incertezas**.

---


# Código

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf
from numpy import linalg as LA

In [ ]:
tickers = [
        'PETR4.SA',     # 3%
        'WEGE3.SA',     # 4%
        'ITUB4.SA',     # 8%
        'BPAC11.SA',    # 27%
        'ITSA4.SA',     # 17%
        'MSFT34.SA',    # 25%
        'NVDC34.SA'     # 16%
            ]


start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime.today()

precos = yf.download(tickers, start=start_date, end=end_date)['Close']
precos

In [ ]:
retornos = precos.pct_change().dropna()
media_retornos = retornos.mean()
matriz_covariancia = retornos.cov()
pesos_carteira = np.array([0.03, 0.04, 0.08, 0.27, 0.17, 0.25, 0.16])
numero_ativos = len(tickers)

pesos_carteira

In [ ]:
numero_simulacoes = 10000
dias_projetados = 252
capital_inicial = 10000

In [ ]:
retorno_medio = retornos.mean(axis=0).to_numpy()
matriz_retornos_medio = retorno_medio * np.ones(shape = (dias_projetados, numero_ativos))

In [ ]:
L = LA.cholesky(matriz_covariancia)
L

In [ ]:
retornos_carteira = np.zeros([dias_projetados, numero_simulacoes])
montante_final = np.zeros(numero_simulacoes)

for s in range (numero_simulacoes):
    Rpdf = np.random.normal(size=(dias_projetados, numero_ativos)) 

    retornos_sinteticos = matriz_retornos_medio + np.inner(Rpdf, L)

    retornos_carteira[:,s] = np.cumprod(np.inner(pesos_carteira, retornos_sinteticos + 1)) * capital_inicial

    montante_final[s] = retornos_carteira[-1,s]

retornos_carteira


    

In [ ]:
plt.plot(retornos_carteira, linewidth=1)
plt.title('Simulação de Monte Carlo - Retornos da Carteira')
plt.xlabel('Dias')
plt.ylabel('Retornos')
plt.grid()
plt.show()

In [ ]:
montante_99 = np.percentile(montante_final, 1)
montante_95 = np.percentile(montante_final, 5)
montante_mediano = np.percentile(montante_final, 50)
cenarios_com_lucro = (len(montante_final[montante_final > capital_inicial]) / len(montante_final)) * 100

print("\n\n📊 Simulação de Monte Carlo:")
print(f"\n🔍 Foram realizadas {numero_simulacoes} simulações para a carteira com os seguintes ativos:")
print(f"\n   ✅ Ativos: {tickers}")
print(f"   ✅ Pesos: {pesos_carteira}")
print(f"\n   💰 Capital inicial: R$ {capital_inicial:,.2f}")
print("\n--------------------------------------------------------------------\n")

print("📈 Resultados das Simulações:")
print(f"\n   ✅ Com 50% de probabilidade, o montante será maior que: R$ {montante_mediano:,.2f}")
print(f"   ✅ Com 95% de probabilidade, o montante será maior que: R$ {montante_95:,.2f}")
print(f"   ✅ Com 99% de probabilidade, o montante será maior que: R$ {montante_99:,.2f}")
print(f"\n   📊 Em {cenarios_com_lucro:.2f}% dos cenários, foi possível obter lucro no próximo 1 ano.")
print("\n--------------------------------------------------------------------\n")

# Níveis de confiança (VAR)

# Cálculo do VaR
var_95 = capital_inicial - np.percentile(montante_final, 5)
var_99 = capital_inicial - np.percentile(montante_final, 1)
# Porcentagem do VaR em relação ao capital inicial
var_95_percent = (var_95 / capital_inicial) * 100
var_99_percent = (var_99 / capital_inicial) * 100

# Exibição dos resultados do VaR
print("📉 Value at Risk (VaR):")
print("\n🔍 O VaR mede a perda máxima esperada em um determinado nível de confiança, considerando condições normais de mercado.")
print(f"\n   ✅ VaR com 95% de confiança -> Risco máximo de R$ {var_95:.2f} ({var_95_percent:.2f}% do capital inicial) <== VAR 95")
print(f"   ✅ VaR com 99% de confiança -> Risco máximo de R$ {var_99:.2f} ({var_99_percent:.2f}% do capital inicial)")
print("\n--------------------------------------------------------------------\n")

# Retorno médio esperado
retorno_esperado = montante_final.mean() - capital_inicial
retorno_esperado_percent = (retorno_esperado / capital_inicial) * 100
print("📈 Retorno Médio Esperado:")
print("\n🔍 O retorno médio esperado da carteira com base nas simulações.")
print(f"\n   💰 Retorno esperado: R$ {retorno_esperado:.2f} ({retorno_esperado_percent:.2f}%)")
print("\n--------------------------------------------------------------------\n")

# Retorno máximo e mínimo
retorno_maximo = montante_final.max()
retorno_minimo = montante_final.min()
retorno_maximo_percent = ((retorno_maximo - capital_inicial) / capital_inicial) * 100
retorno_minimo_percent = ((retorno_minimo - capital_inicial) / capital_inicial) * 100

print("🏆 Retorno Máximo e Mínimo:")
print("\n🔍 Os valores máximos e mínimos simulados para o montante final.")
print(f"\n   🚀 Retorno máximo: R$ {retorno_maximo:.2f} ({retorno_maximo_percent:.2f}%)")
print(f"   📉 Retorno mínimo: R$ {retorno_minimo:.2f} ({retorno_minimo_percent:.2f}%)")
print("\n--------------------------------------------------------------------\n")

# Volatilidade (desvio padrão dos retornos)
volatilidade = montante_final.std()
print("📊 Desvio Padrão dos Retornos (Volatilidade):")
print("\n🔍 A volatilidade mede a dispersão dos retornos simulados, indicando o risco da carteira.")
print(f"\n   📌 Volatilidade dos retornos: R$ {volatilidade:.2f}")
print("\n--------------------------------------------------------------------\n")

# Taxa livre de risco (exemplo: 6% ao ano)
taxa_livre_risco = 0.06
sharpe_ratio = (retorno_esperado / capital_inicial - taxa_livre_risco) / (volatilidade / capital_inicial)
print("📊 Sharpe Ratio:")
print("\n🔍 O Sharpe Ratio mede o retorno ajustado ao risco, comparando o retorno esperado com a volatilidade.")
print(f"\n   📌 Sharpe Ratio: {sharpe_ratio:.2f}")
print("\n--------------------------------------------------------------------\n")

# Cálculo do CVaR (Expected Shortfall)
cvar_95 = capital_inicial - montante_final[montante_final <= np.percentile(montante_final, 5)].mean()
cvar_99 = capital_inicial - montante_final[montante_final <= np.percentile(montante_final, 1)].mean()
# Porcentagem do CVaR em relação ao capital inicial
cvar_95_percent = (cvar_95 / capital_inicial) * 100
cvar_99_percent = (cvar_99 / capital_inicial) * 100

print("📉 CVaR (Conditional Value at Risk) ou Expected Shortfall:")
print("\n🔍 O CVaR mede a perda média esperada nos piores cenários, ou seja, além do VaR. É útil para entender o risco extremo.")
print(f"\n   ✅ CVaR com 95% de confiança -> Risco máximo de: R$ {cvar_95:.2f} ({cvar_95_percent:.2f}% do capital inicial)")
print(f"   ✅ CVaR com 99% de confiança -> Risco máximo de: R$ {cvar_99:.2f} ({cvar_99_percent:.2f}% do capital inicial)")
print("\n--------------------------------------------------------------------\n")


In [ ]:
config = dict(histtype="stepfilled", alpha=0.8, density=False, bins=150, color='blue')
fig, ax = plt.subplots()
ax.hist(montante_final, **config)
# ax.xaxis.set_major_formatter('R${x:,.0f}')
plt.title('Distribuição montantes finais com simulação MC')
plt.xlabel('Montante final (R$)')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Gráfico efeito Cyberpunk

import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")
plt.figure(figsize=(8, 5))

plt.plot(retornos_carteira, linewidth=1)
plt.title('Simulação de Monte Carlo - Retornos da Carteira')
plt.xlabel('Dias')
plt.ylabel('Retornos')
plt.grid()

# # Adiciona efeitos neon ao gráfico
# plt.legend(["Retorno da Carteira"])  # Necessário para aplicar efeito
# mplcyberpunk.add_glow_effects()

plt.show()
